In [4]:

from bit_pred_1_data_collect import fetch_bithumb_candles

import pandas as pd

def preprocess_candles(df: pd.DataFrame) -> pd.DataFrame:
    """
    1) 인덱스를 KST(Asia/Seoul) datetime으로 설정
    2) 결측치는 직전 값으로 보간(Forward Fill)
    3) 1시간 후 종가(target) 컬럼 생성
    """
    df = df.copy()

    # 1) 인덱스를 UTC로 간주하고 KST로 변환
    #    (만약 df.index가 naive datetime이라면 UTC로 로컬라이즈 후 변환)
    df.index = pd.to_datetime(df.index)
    df.index = df.index.tz_localize('UTC').tz_convert('Asia/Seoul')

    # 2) 결측치 처리: 직전 값 보간법
    df.fillna(method='ffill', inplace=True)

    # 3) 레이블 생성: 1시간 뒤 종가
    #    1시간 간격이므로 shift(-1)을 사용
    df['target'] = df['close'].shift(+1)

    return df

# 사용 예시
if __name__ == '__main__':
    # 이미 fetch_bithumb_candles()로 가져온 df가 있다고 가정
    df = fetch_bithumb_candles('BTC_KRW', '1h')

    # 전처리
    df_preprocessed = preprocess_candles(df)

    # 결과 확인
    print(df_preprocessed.head())
    print(df_preprocessed[['close', 'target']].tail())

    # (원한다면) CSV로 저장
    df_preprocessed.to_csv('btc_1h_candles_preprocessed.csv')


                                open      close       high        low  \
timestamp                                                               
2025-05-30 15:00:00+09:00  148341000  148072000  148441000  148044000   
2025-05-30 14:00:00+09:00  148416000  148402000  148600000  148319000   
2025-05-30 13:00:00+09:00  148670000  148401000  148684000  148267000   
2025-05-30 12:00:00+09:00  148502000  148684000  149099000  148439000   
2025-05-30 11:00:00+09:00  147996000  148502000  148751000  147990000   

                                volume     target  
timestamp                                          
2025-05-30 15:00:00+09:00  12.43328597       None  
2025-05-30 14:00:00+09:00  16.85723878  148072000  
2025-05-30 13:00:00+09:00  16.65593647  148402000  
2025-05-30 12:00:00+09:00  29.68742287  148401000  
2025-05-30 11:00:00+09:00   24.5188414  148684000  
                              close    target
timestamp                                    
2024-11-02 16:00:00+09:00  97797

/var/folders/tk/ct0j8j1d0y1ft37qgdzljbzh0000gn/T/ipykernel_9086/918306517.py:19: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
